#### Test Certain Runs
Generate Again a Run using it seed and its properties, saving the lammps files also

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
sys.path.insert(0, './magcolloids')
import magcolloids as mgc
ureg = mgc.ureg # Pint for units

def redefine_field(field,tilt_angle):
    field.fieldx = 'v_Bmag*(sin(v_freq*time*2*PI)*sin(v_theta)+cos(v_theta)*%s)'%np.sin(tilt_angle).magnitude
    field.fieldy = 'v_Bmag*(cos(v_freq*time*2*PI)*sin(v_theta))'%np.sin(tilt_angle).magnitude
    field.fieldz = 'v_Bmag*(cos(v_theta)*%s)'%np.cos(tilt_angle).magnitude
    return field

dir_folder = r"./test_runs/"
name_test = "frequency1_5bistability/"

test_seed = 693999

In [2]:
n_sqrd = 5
num_of_particles = n_sqrd**2
particle_radius = 1.4
enclosure_height = 4
packing_particles = 0.3
tilted_field = False
tilt = 0
magnitude_magfield = 7.2
frequency_magfield = 1.5
angle_magfield = 26.9
stiffness_trap_track = 15e-4
track_particle_id = 10
world_temperature = 300
tot_time = 180

dir_save = dir_folder + name_test
f_nam = "lammps_data"

In [3]:
region, initial_conditions = mgc.initial_setup(num_of_particles, packing = packing_particles, height = enclosure_height, radius = particle_radius)

particles = mgc.particles(
    initial_conditions*ureg.um,
    radius = particle_radius*ureg.um,
    susceptibility = 0.4,
    diffusion=0.07*ureg.um**2/ureg.s,
    density = 1e3*ureg.kg/ureg.m**3,
    temperature=world_temperature*ureg.K)

trap_track = mgc.bistable_trap(
                initial_conditions[track_particle_id:track_particle_id+1,:]*ureg.um,
                np.array([[1,0,0]]),
                particles, subsets = [track_particle_id],
                distance = 0*ureg.um,
                height = 0 * ureg.pN*ureg.nm,
                stiffness = stiffness_trap_track*ureg.pN/ureg.nm)

field = mgc.field(magnitude = magnitude_magfield*ureg.mT, frequency = frequency_magfield*ureg.Hz, angle = angle_magfield*ureg.degrees)
if tilted_field:
    field = redefine_field(field,tilt*ureg.deg)
    
world = mgc.world(particles, [trap_track], temperature = world_temperature*ureg.K,
                region=region*ureg.um, boundaries = ['p','p','f'], walls = [False,False,True],
                dipole_cutoff = 20*ureg.um)

sim = mgc.sim(dir_name = dir_save, file_name = f_nam,
        timestep = 1e-4*ureg.s, framerate = 30*ureg.Hz, total_time = tot_time*ureg.s,
        particles = particles, traps = [trap_track], world = world, field = field,
        output = ["x","y","z"])

sim.seed = test_seed

In [4]:
sim.generate_scripts()
sim.run()

CompletedProcess(args=['c:\\Users\\emino\\OneDrive\\Desktop\\MagColloids\\magcolloids-master\\magcolloids\\lammps_executables\\lmp_mingw64-native.exe', '-in', '/test_runs/frequency1_5bistability/lammps_data.lmpin'], returncode=0, stdout=b'LAMMPS (18 Jun 2019)\r\nReading data file ...\r\n  orthogonal box = (-11.5 -11.5 -2) to (11.5 11.5 2)\r\n  1 by 1 by 1 MPI processor grid\r\n  reading atoms ...\r\n  26 atoms\r\n  scanning bonds ...\r\n  1 = max bonds/atom\r\n  reading bonds ...\r\n  1 bonds\r\nFinding 1-2 1-3 1-4 neighbors ...\r\n  special bond factors lj:   0          0          0         \r\n  special bond factors coul: 0          0          0         \r\n  1 = max # of 1-2 neighbors\r\n  0 = max # of 1-3 neighbors\r\n  0 = max # of 1-4 neighbors\r\n  1 = max # of special neighbors\r\n  special bonds CPU = 0 secs\r\n  read_data CPU = 0.0384369 secs\r\n25 atoms in group Atoms\r\n1 atoms in group Traps\r\nNeighbor list info ...\r\n  update every 1 steps, delay 1 steps, check yes\r\n 